# [한국어 데이터로 챗봇 만들기]

In [1]:
# 라이브러리 확인
import tensorflow
print(tensorflow.__version__)

2.6.0


In [2]:
import tensorflow_datasets as tfds
import os
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# 현재 위치 확인
!pwd

/aiffel/aiffel/13.NLP


# 1. 데이터 수집
- 한국어 챗봇 데이터 by.송영숙
(https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv)


In [4]:
# csv 파일 저장 경로
file_path = '/aiffel/aiffel/13.NLP/ChatbotData .csv'

In [5]:
data = pd.read_csv(file_path)

In [6]:
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


# 2. 데이터 전처리
- 공백, 특수문자 처리
- 토크나이징
- 병렬 데이터 구축

In [7]:
MAX_SAMPLES = 10000
print(MAX_SAMPLES)

10000


In [8]:
# 정규 표현식 사용하여 토크나이징을 위한 전처리
def preprocess_sentence(sentence):

  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)   # 특수문자 양옆에 공백 추가
  sentence = re.sub(r'[" "]+', " ", sentence)         # 연속된 공백을 단일 공백으로 변환
  sentence = re.sub(r"[^가-힣?.!,]+", " ", sentence)  # 한글 제외한 나머지 문자 공백으로 대체
  sentence = sentence.strip()   # 양쪽 공백 제거
  return sentence

print("슝=3")

슝=3


In [9]:
# 질문과 답변 전처리
data['Q'] = data['Q'].apply(preprocess_sentence)
data['A'] = data['A'].apply(preprocess_sentence)

In [10]:
# 토크나이징 및 병렬 데이터 구축
questions = data['Q'].tolist()
answers = data['A'].tolist()

In [11]:
def tokenize_and_pad(sentences, max_len):
    sequences = tokenizer.texts_to_sequences(sentences)
    return pad_sequences(sequences, maxlen=max_len, padding='post')

# 최대 길이 설정
MAX_LEN = 20

# 토크나이징 및 패딩
questions = tokenize_and_pad(data['Q'], MAX_LEN)
answers = tokenize_and_pad(data['A'], MAX_LEN)

NameError: name 'tokenizer' is not defined

In [21]:
# 데이터 확인
for i in range(3):
    print(f'질문: {questions[i]}')
    print(f'답변: {answers[i]}')

질문: 시 땡 !
답변: 하루가 또 가네요 .
질문: 지망 학교 떨어졌어
답변: 위로해 드립니다 .
질문: 박 일 놀러가고 싶다
답변: 여행은 언제나 좋죠 .


# 3. SubwordTextEncoder 사용하기

In [22]:
# 질문과 답변 데이터셋에 대해서 Vocabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
print("슝=3 ")

슝=3 


In [23]:
# 디코더의 문장 생성 과정에서 사용할 '시작&종료 토큰'에 대해서도 임의로 단어장에 추가하여 정수 부여
# 이미 생성된 단어장 번호와 겹치지 않도록 각 단어장의 크기 +1로 번호 부여

# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
print("슝=3")

슝=3


In [24]:
# 시작, 종료 토큰에 부여된 정수 출력하여 확인
print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

START_TOKEN의 번호 : [8127]
END_TOKEN의 번호 : [8128]


In [25]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8129


### 각 단어 고유한 정수로 인코딩 & 패딩


In [37]:
# 인코딩
questions = [START_TOKEN + tokenizer.encode(sentence) + END_TOKEN for sentence in questions]
answers = [START_TOKEN + tokenizer.encode(sentence) + END_TOKEN for sentence in answers]

# 패딩
MAX_LENGTH = 40
questions = tf.keras.preprocessing.sequence.pad_sequences(questions, maxlen=MAX_LENGTH, padding='post')
answers = tf.keras.preprocessing.sequence.pad_sequences(answers, maxlen=MAX_LENGTH, padding='post')

# 데이터셋 생성
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((questions, answers))
dataset = dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

TypeError: Expected binary or unicode string, got array([8127, 4188, 3045,   42, 8128,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [32]:
# # 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
# def tokenize_and_filter(inputs, outputs):
#   tokenized_inputs, tokenized_outputs = [], []
  
#   for (sentence1, sentence2) in zip(inputs, outputs):
#     # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
#     sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
#     sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

#     # 최대 길이 40 이하인 경우에만 데이터셋으로 허용
#     if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
#       tokenized_inputs.append(sentence1)
#       tokenized_outputs.append(sentence2)
  
#   # 최대 길이 40으로 모든 데이터셋을 패딩
#   tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
#       tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
#   tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
#       tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
#   return tokenized_inputs, tokenized_outputs
# print("슝=3")

슝=3


# 4. 모델 구성(트랜스포머)


In [33]:
from tensorflow.keras.layers import Input, Dense, Embedding, MultiHeadAttention, LayerNormalization, Dropout
from tensorflow.keras.models import Model

In [ ]:
def transformer_model(vocab_size, num_layers, units, d_model, num_heads, dropout):
    inputs = Input(shape=(None,))
    dec_inputs = Input(shape=(None,))

    enc_padding_mask = None
    look_ahead_mask = None
    dec_padding_mask = None

    # 인코더 임베딩
    embeddings = Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = Dropout(dropout)(embeddings)

    # 디코더 임베딩
    dec_embeddings = Embedding(vocab_size, d_model)(dec_inputs)
    dec_embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    dec_embeddings = Dropout(dropout)(dec_embeddings)

    for _ in range(num_layers):
        # 인코더
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(embeddings, embeddings, embeddings, attention_mask=enc_padding_mask)
        attn_output = Dropout(dropout)(attn_output)
        out1 = LayerNormalization(epsilon=1e-6)(attn_output + embeddings)

        # 디코더
        attn_output_1 = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(dec_embeddings, dec_embeddings, dec_embeddings, attention_mask=look_ahead_mask)
        attn_output_1 = Dropout(dropout)(attn_output_1)
        out2 = LayerNormalization(epsilon=1e-6)(attn_output_1 + dec_embeddings)

        attn_output_2 = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(out2, out1, out1, attention_mask=dec_padding_mask)
        attn_output_2 = Dropout(dropout)(attn_output_2)
        out3 = LayerNormalization(epsilon=1e-6)(attn_output_2 + out2)

        embeddings = out1
        dec_embeddings = out3

    outputs = Dense(vocab_size)(dec_embeddings)

    return Model(inputs=[inputs, dec_inputs], outputs=outputs)

# 모델 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1

model = transformer_model(
    VOCAB_SIZE, NUM_LAYERS, UNITS, D_MODEL, NUM_HEADS, DROPOUT)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

In [34]:
# # 트랜스포머 함수 정의
# def transformer(vocab_size,
#                 num_layers,
#                 units,
#                 d_model,
#                 num_heads,
#                 dropout,
#                 name="transformer"):
#   inputs = tf.keras.Input(shape=(None,), name="inputs")
#   dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

#   # 인코더에서 패딩을 위한 마스크
#   enc_padding_mask = tf.keras.layers.Lambda(
#       create_padding_mask, output_shape=(1, 1, None),
#       name='enc_padding_mask')(inputs)

#   # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
#   # 내부적으로 패딩 마스크도 포함되어져 있습니다.
#   look_ahead_mask = tf.keras.layers.Lambda(
#       create_look_ahead_mask,
#       output_shape=(1, None, None),
#       name='look_ahead_mask')(dec_inputs)

#   # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
#   # 디코더에서 패딩을 위한 마스크
#   dec_padding_mask = tf.keras.layers.Lambda(
#       create_padding_mask, output_shape=(1, 1, None),
#       name='dec_padding_mask')(inputs)

#   # 인코더
#   enc_outputs = encoder(
#       vocab_size=vocab_size,
#       num_layers=num_layers,
#       units=units,
#       d_model=d_model,
#       num_heads=num_heads,
#       dropout=dropout,
#   )(inputs=[inputs, enc_padding_mask])

#   # 디코더
#   dec_outputs = decoder(
#       vocab_size=vocab_size,
#       num_layers=num_layers,
#       units=units,
#       d_model=d_model,
#       num_heads=num_heads,
#       dropout=dropout,
#   )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

#   # 완전연결층
#   outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

#   return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)
# print("슝=3")

슝=3


In [35]:
# 모델 생성
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

NameError: name 'create_padding_mask' is not defined

In [ ]:
# 손실함수
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)
print("슝=3")

In [ ]:
# 커스텀된 학습률
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
print("슝=3")

In [ ]:
# 방금 정의한 커스텀 학습률 스케쥴링 계획 시각화
sample_learning_rate = CustomSchedule(d_model=128)

plt.plot(sample_learning_rate(tf.range(200000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

In [ ]:
# 모델 컴파일
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
print("슝=3")

In [ ]:
# 훈련
EPOCHS = 10
model.fit(dataset, epochs=EPOCHS, verbose=1)

# 5. 모델 평가
[예측 단계 프로세스]
1. 새로운 입력 문장에 대해서는 훈련 때와 동일한 전처리를 거친다.
2. 입력 문장을 토크나이징하고, START_TOKEN과 END_TOKEN을 추가한다.
3. 패딩 마스킹과 룩 어헤드 마스킹을 계산한다.
4. 디코더는 입력 시퀀스로부터 다음 단어를 예측한다.
5. 디코더는 예측된 다음 단어를 기존의 입력 시퀀스에 추가하여 새로운 입력으로 사용한다.
6. END_TOKEN이 예측되거나 문장의 최대 길이에 도달하면 디코더는 동작을 멈춘다.

In [ ]:
# 위 과정을 담은 함수 정의
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)
print("슝=3")

In [ ]:
# 임의의 입력 문장에 대해 decoder_inference() 함수 호출하여 챗봇의 대답을 얻는 함수 정의
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence
print("슝=3")

In [ ]:
# 임의 문장으로 테스트
sentence_generation('집에 가고 싶어!')